## Importing Libraries

In [1]:
# Load libraries
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords, wordnet
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore', category = DeprecationWarning)
import re
import string
import time
# helpful character encoding module
import chardet
import seaborn as sns
from tqdm import tqdm
import collections
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import gensim

In [2]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\spiky\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\spiky\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\spiky\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\spiky\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from wordcloud import WordCloud
from pprint import pprint
import pickle 
import pyLDAvis
import os
import pyLDAvis.gensim_models
import stylecloud

## Load the dataset

In [4]:
df = pd.read_csv("lda_cleaned_dataset.csv")
df.head()

C:\Users\spiky\anaconda3\envs\python37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,id,song,album,artist,acousticness_x,danceability_x,duration_ms_x,energy_x,instrumentalness_x,...,readability,grade,richness,Language,neg,neu,pos,compound,compound_vader_polarity,artist_popularity
0,0,1mTxphc9naXMLE9qfMkEVh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-5.036,College Graduate,0.177977,en,0.102,0.846,0.052,-0.9857,-1.0,0.000000
1,1,2Tp1yCGNjjZ39HTEnKaqhx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,90.502,5th Grade,0.076503,en,0.294,0.455,0.251,-0.8895,-1.0,5.000000
2,2,3plzStfrkTOq9lehqqI3bG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,33.028,College Level,0.254582,en,0.064,0.800,0.136,0.9856,1.0,46.000000
3,3,2MGhUOnAifh3znlInWnywc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-61.661,College Graduate,0.191257,en,0.109,0.817,0.074,-0.5831,-1.0,38.000000
4,4,1vDRvO1X021HlJY2NnqUU2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69.626,9th Grade,0.200000,en,0.034,0.913,0.053,0.6788,1.0,60.848402


In [5]:
df.shape

C:\Users\spiky\anaconda3\envs\python37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(66452, 60)

In [6]:
df['lyrics'] = df['lyrics'].astype('str')

C:\Users\spiky\anaconda3\envs\python37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
def convert_year(x):
    
    if x >= 1930 and x < 1940:
        
        label = '1930-40'
        
    elif x >= 1940 and x < 1950:
        
        label = '1940-50'
    
    elif x >= 1950 and x < 1960:
        
        label = '1950-60'
        
    elif x >= 1960 and x < 1970:
        
        label = '1960-70'
        
    elif x >= 1970 and x < 1980:
        
        label = '1970-80'
        
    elif x >= 1980 and x < 1990:
        
        label = '1980-90'
        
    elif x >= 1990 and x < 2000:
        
        label = '1990-2000'
        
    elif x >= 2000 and x < 2010:
        
        label = '2000-10'
        
    elif x >= 2010:
        
        label = '2010-now'
        
    elif x < 1930:
        label = 'pre-1930'
        
    return label

C:\Users\spiky\anaconda3\envs\python37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
df['time_period'] = df['year'].apply(convert_year)

C:\Users\spiky\anaconda3\envs\python37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
df['time_period'].value_counts()

C:\Users\spiky\anaconda3\envs\python37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1990-2000    11710
1980-90      11310
1970-80      10901
2010-now     10439
1960-70       8951
2000-10       7744
1950-60       3897
1940-50        679
1930-40        541
pre-1930       280
Name: time_period, dtype: int64

In [16]:
popular_songs = df[df['popularity_dummy'] == 1]

C:\Users\spiky\anaconda3\envs\python37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
popular_songs['time_period'].value_counts()

C:\Users\spiky\anaconda3\envs\python37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1990-2000    5641
1970-80      5555
1980-90      5335
2000-10      4053
2010-now     3733
1960-70      2939
1950-60       122
1940-50        22
1930-40         6
Name: time_period, dtype: int64